<a href="https://colab.research.google.com/github/PavlosPo/social-networks/blob/pavlos-playground/nx-cugraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

You can check the output of `!nvidia-smi` to check which GPU you have.  Please uncomment the cell below if you'd like to do that.  Currently, RAPIDS runs on all available Colab GPU instances.

In [ ]:
# !nvidia-smi

Thu Nov 30 17:46:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Setup:
This set up script:

1. Checks to make sure that the GPU is RAPIDS compatible
1. Installs the **current stable version** of RAPIDSAI's core libraries using pip, which are:
  1. cuDF
  1. cuML
  1. cuGraph
  1. cuSpatial
  1. cuxFilter
  1. cuCIM
  1. xgboost

**This will complete in about 5-6 minutes**

If you require installing the **nightly** releases of RAPIDSAI, please use the [RAPIDS Conda Colab Template notebook](https://colab.research.google.com/drive/1TAAi_szMfWqRfHVfjGSqnGVLr_ztzUM9) and use the nightly parameter option when running the RAPIDS installation cell.


In [1]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py
!pip install networkx ogb


Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 481, done.
remote: Counting objects: 100% (212/212), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 481 (delta 143), reused 124 (delta 91), pack-reused 269
Receiving objects: 100% (481/481), 133.58 KiB | 1.04 MiB/s, done.
Resolving deltas: 100% (245/245), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.3 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has a Tesla T4 GPU!
We will install the latest stable RAPIDS via pip 24.4.*!  Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 976.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 GB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 MB 7.

# RAPIDS is now installed on Colab.  
You can copy your code into the cells below or use the below to validate your RAPIDS installation and version.  
# Enjoy!

In [2]:
import cudf
cudf.__version__

'24.04.01'

In [3]:
import cuml
cuml.__version__

'24.04.00'

In [7]:
import cugraph
cugraph.__version__

'24.04.00'

In [5]:
import cuspatial
cuspatial.__version__

'24.04.00'

In [6]:
import cuxfilter
cuxfilter.__version__

'24.04.01'

# Next Steps #

For an overview of how you can access and work with your own datasets in Colab, check out [this guide](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92).

For more RAPIDS examples, check out our RAPIDS notebooks repos:
1. https://github.com/rapidsai/notebooks
2. https://github.com/rapidsai/notebooks-contrib

# Betweeness

In [ ]:
# %load_ext cudf.pandas
# pandas API is now GPU accelerated

import pandas as pd
import networkx as nx
import cugraph as cg
import cudf
from cugraph.experimental import PropertyGraph
from ogb.linkproppred import LinkPropPredDataset
dataset = LinkPropPredDataset(name='ogbl-ppa')

split_edge = dataset.get_edge_split()
train_edge, valid_edge, test_edge = split_edge["train"], split_edge["valid"], split_edge["test"]
graph = dataset[0]  # graph: library-agnostic graph object

G = nx.Graph()

for i in range(graph['num_nodes']):
    source = graph['edge_index'][0][i]
    target = graph['edge_index'][1][i]
    G.add_edge(source, target)
# Calculate betweenness centrality
bc_scores = cg.betweenness_centrality(G)

# Save in pd.DataFrame
results_df = pd.DataFrame(bc_scores.items(), columns=['Node', 'Betweenness Centrality'])

# Save it in csv
results_df.to_csv('betweenness_centrality.csv', index=False)


# Closeness Centrality

In [ ]:
import cugraph as cg
import cudf
import pandas as pd
from ogb.linkproppred import LinkPropPredDataset

print('Loading dataset...')
dataset = LinkPropPredDataset(name='ogbl-ppa')

split_edge = dataset.get_edge_split()
train_edge, valid_edge, test_edge = split_edge["train"], split_edge["valid"], split_edge["test"]
graph = dataset[0]  # graph: library-agnostic graph object

print('Creating graph...')
# Extract edge data
edges_df = pd.DataFrame({'source': graph['edge_index'][0], 'target': graph['edge_index'][1]})
edges_cudf = cudf.from_pandas(edges_df)

# Initialize CuGraph graph directly from edge list
G = cg.Graph()
G.from_cudf_edgelist(edges_cudf, source='source', destination='target')

print('Calculating closeness centrality...')
# Calculate closeness centrality
cc_scores = cg.closeness_centrality(G)

# Convert closeness centrality scores to dictionary
results = cc_scores.to_pandas().set_index('vertex')['closeness_centrality'].to_dict()

# Save in pd.DataFrame
print('Saving results...')
results_df = pd.DataFrame(results.items(), columns=['Node', 'Closeness Centrality'])

# Save results in csv
results_df.to_csv('closeness_centrality.csv', index=False)

print('Done!')


ModuleNotFoundError: No module named 'cugraph'

# Louvain Communities

In [ ]:
import cugraph as cg
import cudf
import networkx as nx
import pandas as pd
from ogb.linkproppred import LinkPropPredDataset

print('Loading dataset...')
dataset = LinkPropPredDataset(name='ogbl-ppa')

split_edge = dataset.get_edge_split()
train_edge, valid_edge, test_edge = split_edge["train"], split_edge["valid"], split_edge["test"]
graph = dataset[0]  # graph: library-agnostic graph object

print('Creating graph...')
# Initialize CuGraph graph directly from edge list
G = nx.Graph()

for i in range(graph['num_nodes']):
    source = graph['edge_index'][0][i]
    target = graph['edge_index'][1][i]
    G.add_edge(source, target)

print('Calculating Louvain Communities...')
# Calculate katz centrality
cc_scores = nx.community.louvain_communities(G, backend='cugraph')

# Convert communities to a list of lists for easier handling
communities_list = [list(community) for community in cc_scores]

# Save communities to CSV
df = pd.DataFrame({'Community': range(len(communities_list))})
df['Nodes'] = communities_list
df.to_csv('louvain_communities.csv', index=False)

# Create a DataFrame with each node mapped to its community ID
print('Saving results...')
nodes_df = pd.DataFrame([(idx, node) for idx, nodes in enumerate(communities_list) for node in nodes], columns=['CommunityID', 'NodeID'])

# Save the DataFrame to a CSV file
nodes_df.to_csv('louvain_communities_format_2.csv', index=False)
print('Done!')

# Leiden Communities

In [35]:
import cugraph as cg
import cudf
import networkx as nx
import pandas as pd
from ogb.linkproppred import LinkPropPredDataset

print('Loading dataset...')
dataset = LinkPropPredDataset(name='ogbl-ppa')

split_edge = dataset.get_edge_split()
train_edge, valid_edge, test_edge = split_edge["train"], split_edge["valid"], split_edge["test"]
graph = dataset[0]  # graph: library-agnostic graph object

print('Creating graph...')
# Initialize CuGraph graph directly from edge list
G = nx.Graph()

for i in range(graph['num_nodes']):
    source = graph['edge_index'][0][i]
    target = graph['edge_index'][1][i]
    G.add_edge(source, target)

print('Calculating Leiden Communities...')
# Calculate Leiden Communities
parts, modularity_score = cg.community.leiden(G)

# Save communities to CSV
df = pd.DataFrame([(node_id, partition_id) for node_id, partition_id in parts.items()], columns=['NodeID', 'PartitionID'])
df.to_csv('leiden_communities.csv', index=False)

Loading dataset...
Creating graph...
Calculating Leiden Communities...


/usr/local/lib/python3.10/dist-packages/cugraph/structure/symmetrize.py:93: FutureWarning: Multi is deprecated and the removal of multi edges will no longer be supported from 'symmetrize'. Multi edges will be removed upon creation of graph instance.
  warnings.warn(
